In [ ]:
####OPTION A######################

In [ ]:
# ==========================================
# OPTION A — Multi-cell soma segmentation for Ch2
# - Uses physical (µm) params, prunes thin neurite bridges (opening),
#   and seeds watershed from thick soma cores (distance >= soma_min_radius).
# - Keeps your lysosome (Ch1) blob detection & all CSV/video/fig outputs.
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion  # if you want optional tweaks

# ==========================================
# CONFIG
# ==========================================
# Replace with your file. Two examples:
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_4 Airy-CBs_300425.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# OPTION A: physical (µm) params → voxel units
# ==========================================
soma_min_radius_um = 3.0       # expected minimal soma radius (µm)  ⟵ tune 2.5–6
neurite_max_radius_um = 1.0    # anything thinner than this is "bridge"  ⟵ tune 0.6–1.5
smooth_sigma_um = 0.7          # light denoise in µm                 ⟵ tune 0.5–1.0

# Convert to voxel units (per-axis for smoothing; isotropic ball for morphology)
soma_min_radius_vox   = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_max_radius_vox = max(1, int(round(neurite_max_radius_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_max_radius_vox={neurite_max_radius_vox}, smooth_sigma_vox={smooth_sigma_vox}")

# ==========================================
# 2) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)

# LoG returns sigma; convert to a scale radius in index coords; for 3D use sqrt(3)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

# --- Convert per-lysosome metrics to physical units (µm) ---
if len(blobs) > 0:
    # positions in µm
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um

    # single radius/diameter/volume per lysosome (equivalent-sphere), in µm
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5  # keep your original convention
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

# --- Region bins (optional export kept) ---
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- Per-blob DF (µm only) ---
blob_ids = np.arange(1, len(blobs) + 1)

df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
    # "region_id": region_labels
})

# --- Save (µm-only) ---
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 3) Viewer base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE) — OPTION A
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# 3D Gaussian using per-axis sigmas (handles anisotropy)
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# Base neuron "foreground" via local threshold per z (illumination-robust)
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    # smaller window + gentler offset reduces over-connection
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Remove tiny specks
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)

# **Key change**: OPEN (remove thin bridges) instead of CLOSE (which connects somas)
neuron_mask = binary_opening(neuron_mask, ball(neurite_max_radius_vox))

# Optional: very light closing if opening nibbled soma rims
# neuron_mask = binary_closing(neuron_mask, ball(1))

# Distance inside the neuron
dist = edt(neuron_mask)

# Soma-only core mask: keep voxels thicker than soma_min_radius_vox
cell_core = dist >= soma_min_radius_vox

# Clean seeds (avoid fragments)
cell_core = binary_opening(cell_core, ball(1))
cell_core = remove_small_objects(cell_core, min_size=2000, connectivity=3)

# Label seeds: one marker per soma
markers = label(cell_core, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds.")

# Watershed territories across the full neuron, seeded by soma cores
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels (core):", int(cell_core.sum()))

# ==========================================
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table (µm-only metrics)
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 5b) Per-cell (Ch2) volumes (already in µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE + seeds)
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma core seeds (helpful QA)
try:
    core_layer = viewer.add_labels(markers.astype(np.uint16), name='Soma core seeds', opacity=0.6)
    core_layer.blending = 'translucent_no_depth'
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),  # size in index units for display
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option A (soma seeds + neurite opening)"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Save a 3D screenshot (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option A"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 9) Run viewer
# ==========================================
napari.run()

In [ ]:
############new vwrsion OPTION A##################################3

In [1]:
# ==========================================
# OPTION A — Multi-cell soma segmentation for Ch2
# - Uses physical (µm) params, prunes thin neurite bridges (opening),
#   and seeds watershed from thick soma cores (distance >= soma_min_radius).
# - Keeps your lysosome (Ch1) blob detection & all CSV/video/fig outputs.
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries
from skimage.morphology import binary_erosion  # if you want optional tweaks

# ==========================================
# CONFIG
# ==========================================
# Replace with your file. Two examples:
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_3 Airy-CBs_300425.czi"
#file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/images/40A_UAS-TMEM192-3xHA x 40A 71G10 MARCM_around 12h - for quantification_4 Airy-CBs_300425.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# OPTION A: physical (µm) params → voxel units
# ==========================================
soma_min_radius_um = 3.0       # expected minimal soma radius (µm)  ⟵ tune 2.5–6
neurite_max_radius_um = 1.0    # anything thinner than this is "bridge"  ⟵ tune 0.6–1.5
smooth_sigma_um = 0.7          # light denoise in µm                 ⟵ tune 0.5–1.0

# Convert to voxel units (per-axis for smoothing; isotropic ball for morphology)
soma_min_radius_vox   = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_max_radius_vox = max(1, int(round(neurite_max_radius_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_max_radius_vox={neurite_max_radius_vox}, smooth_sigma_vox={smooth_sigma_vox}")

# ==========================================
# 2) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)

# LoG returns sigma; convert to a scale radius in index coords; for 3D use sqrt(3)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

# --- Convert per-lysosome metrics to physical units (µm) ---
if len(blobs) > 0:
    # positions in µm
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um

    # single radius/diameter/volume per lysosome (equivalent-sphere), in µm
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5  # keep your original convention
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

# --- Region bins (optional export kept) ---
num_bins = (4, 4, 4)
z_bins = np.linspace(0, image.shape[0], num_bins[0] + 1, dtype=int)
y_bins = np.linspace(0, image.shape[1], num_bins[1] + 1, dtype=int)
x_bins = np.linspace(0, image.shape[2], num_bins[2] + 1, dtype=int)

if len(blobs) > 0:
    z_idx = np.clip(np.digitize(blobs[:, 0], z_bins) - 1, 0, num_bins[0]-1)
    y_idx = np.clip(np.digitize(blobs[:, 1], y_bins) - 1, 0, num_bins[1]-1)
    x_idx = np.clip(np.digitize(blobs[:, 2], x_bins) - 1, 0, num_bins[2]-1)
    region_labels = z_idx * (num_bins[1] * num_bins[2]) + y_idx * num_bins[2] + x_idx
else:
    region_labels = np.array([], dtype=int)

# --- Per-blob DF (µm only) ---
blob_ids = np.arange(1, len(blobs) + 1)

df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
    # "region_id": region_labels
})

# --- Save (µm-only) ---
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 3) Viewer base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 4) CH2: segmentation (CELL vs OUTSIDE) — OPTION A
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# 3D Gaussian using per-axis sigmas (handles anisotropy)
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# Base neuron "foreground" via local threshold per z (illumination-robust)
neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    # smaller window + gentler offset reduces over-connection
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Remove tiny specks
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)

# **Key change**: OPEN (remove thin bridges) instead of CLOSE (which connects somas)
neuron_mask = binary_opening(neuron_mask, ball(neurite_max_radius_vox))

# Optional: very light closing if opening nibbled soma rims
# neuron_mask = binary_closing(neuron_mask, ball(1))

# Distance inside the neuron
dist = edt(neuron_mask)

# Soma-only core mask: keep voxels thicker than soma_min_radius_vox
cell_core = dist >= soma_min_radius_vox

# Clean seeds (avoid fragments)
cell_core = binary_opening(cell_core, ball(1))
cell_core = remove_small_objects(cell_core, min_size=2000, connectivity=3)

# Label seeds: one marker per soma
markers = label(cell_core, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds.")

# Watershed territories across the full neuron, seeded by soma cores
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels (core):", int(cell_core.sum()))

# ==========================================
# 5) Map lysosomes to 2 classes (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    # Per-class counts (2 classes)
    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    # Per-cell counts (only those inside 'cell')
    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    # Full table (µm-only metrics)
    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# Define cells that contain lysosomes (used later for territory stamps)
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 5b) Per-cell (Ch2) volumes (already in µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 6) Visualization (CELL vs OUTSIDE + seeds)
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma core seeds (helpful QA)
try:
    core_layer = viewer.add_labels(markers.astype(np.uint16), name='Soma core seeds', opacity=0.6)
    core_layer.blending = 'translucent_no_depth'
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan for cell
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white for outside

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),  # size in index units for display
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option A (soma seeds + neurite opening)"
except Exception:
    pass

# EXTRA: Labels for ALL cells (with vs without lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 7) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (ch2 * 255).astype(np.uint8)
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 8) Save a 3D screenshot (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    """Save a 3D screenshot looking down the Z axis (XY plane)."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    """Save a 3D screenshot looking along ±X so the YZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)   # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    """Save a 3D screenshot looking along ±Y so the XZ plane is visible."""
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)    # (elevation, azimuth, roll)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

# Switch to 3D and set window title, then save XY/YZ/XZ shots
viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option A"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 10) EXTRA: Export Original .CZI as MP4 (RAW z-stack)
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which raw channel to export:
    # raw_stack = image        # Ch1 (lysosomes)
    raw_stack = image_2        # Ch2 (neurons)

    raw_u8 = _to_uint8(raw_stack)

    frames = []
    for z in range(raw_u8.shape[0]):
        frame = cv2.cvtColor(raw_u8[z], cv2.COLOR_GRAY2BGR)
        frames.append(frame)

    out_raw = "original_czi_raw.mp4"
    imageio.mimsave(out_raw, frames, fps=8, format="FFMPEG")
    print(f"Saved: {out_raw}")

except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 11) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # LEFT panel (raw). You can switch to Ch1 if preferred:
    # left_u8 = _to_uint8(image)      # Ch1 raw (lysosomes)
    left_u8 = _to_uint8(image_2)      # Ch2 raw (neurons)

    # RIGHT panel base = segmented channel (Ch2)
    right_base_u8 = _to_uint8(image_2)

    Z, H, W = left_u8.shape
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=8, format="FFMPEG")

    for z in range(Z):
        # ----- LEFT (raw) -----
        left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)

        # ----- RIGHT (seg overlay on Ch2) -----
        base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)

        # Overlay cell mask in green
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # Draw lysosome circles (yellow if inside cell, white if outside)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    inside = ('cell_mask' in globals() and cell_mask[z, y, x])
                    color = (255, 255, 0) if inside else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # Concat panels with a thin divider
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg is unavailable
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(left_u8.shape[0]):
            left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W:
                        color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                        cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 9) Run viewer
# ==========================================
napari.run()

C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\nahue\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Raw CZI shape: (2, 31, 748, 748)
Voxel size (µm): X=0.0458, Y=0.0458, Z=0.21  |  equiv linear µm=0.07608
soma_min_radius_vox=39, neurite_max_radius_vox=13, smooth_sigma_vox=(3.3333333333333335, 15.284786630997159, 15.284786630997159)
Detected 365 lysosomes.
Saved: lysosome_blobs_regions.csv (µm-only)
Detected 0 soma seeds.
neuron voxels: 30709
cell voxels (core): 0
Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, lysosomes_with_cell_vs_outside.csv (µm-only metrics)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (748, 748) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved: ch2_fused_cell.mp4
Saved 3D XY screenshot: cells_segmentation_lysosomes_XY_3d.png
Saved 3D YZ screenshot: cells_segmentation_lysosomes_YZ_3d.png
Saved 3D XZ screenshot: cells_segmentation_lysosomes_XZ_3d.png


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (748, 748) to (752, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved: original_czi_raw.mp4


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 748) to (1504, 752) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved: raw_vs_segmented_side_by_side.mp4


In [ ]:
###OPTION B#############

In [ ]:
# ==========================================
# OPTION B — Neurite-suppressed soma segmentation for Ch2
# - Suppress thin neurites via white-tophat subtraction
# - Seed watershed from h-maxima on thickness (distance map)
# - Uses physical (µm) params; maps lysosomes to per-cell territories
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    white_tophat, h_maxima
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Pick your file (uncomment the one you want)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (physical, then converted to voxels)
# ==========================================
# Soma scale and neurite suppression
soma_min_radius_um     = 3.0    # minimal soma radius for seed (µm). ↑ if two somas merge
neurite_tophat_um      = 0.9    # remove bright tubes/thin bridges up to ~this radius (µm)
tophat_strength        = 1.25   # how much of the thin structures to subtract (1.0–1.5)
smooth_sigma_um        = 0.7    # light denoise before thresholding (µm)

# h-maxima for seed stabilization (fraction of soma thickness)
h_fraction_of_radius   = 0.4    # 0.3–0.6; ↑ to merge bumpy peaks within one soma

# Convert to voxel units
soma_min_radius_vox   = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_tophat_vox    = max(1, int(round(neurite_tophat_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
h_vox = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))

print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_tophat_vox={neurite_tophat_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, h_vox={h_vox}")

# ==========================================
# 3) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune if needed
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

blob_ids = np.arange(1, len(blobs) + 1)
df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) CH2: neurite-suppressed segmentation (CELL vs OUTSIDE)
# ==========================================
# Normalize Ch2
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light 3D Gaussian (accounts for anisotropy)
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# ---- Neurite suppression via white-tophat ----
# Extract thin bright structures (neurites) and subtract them
#thin_struct = white_tophat(ch2, selem=ball(neurite_tophat_vox))
thin_struct = white_tophat(ch2, footprint=ball(neurite_tophat_vox))
enhanced = ch2 - tophat_strength * thin_struct
enhanced = np.clip(enhanced, 0.0, 1.0)

# ---- Foreground (neuron) mask via adaptive per-slice threshold ----
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Remove tiny specks; lightly restore soma rims if needed
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))  # tiny fill without bridging

# ---- Thickness map & h-maxima seeding ----
dist = edt(neuron_mask)

# Keep only thick voxels for seed candidate region
thick = dist >= soma_min_radius_vox

# Stabilize peaks within each soma so we get 1 seed per soma (or per nucleus)
# h is in voxels; use ~40% of soma radius
h = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))
maxima = h_maxima(dist, h=h) & thick

# Clean and label seeds
maxima = remove_small_objects(maxima, min_size=500, connectivity=3)
markers = label(maxima, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (h-maxima).")

# ---- Watershed territories across the neuron ----
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("seed voxels (maxima):", int(maxima.sum()))

# ==========================================
# 6) Map lysosomes to (cell / outside) + per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 7) Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 8) Visualization
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma seeds (maxima) layer for QA
try:
    seeds_layer = viewer.add_labels(label(maxima).astype(np.uint16), name='Soma seeds (h-maxima)', opacity=0.6)
    seeds_layer.blending = 'translucent_no_depth'
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option B (tophat neurite suppression + h-maxima)"
except Exception:
    pass

# EXTRA: territory stamps (all cells, yellow if contains lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 9) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # use neurite-suppressed view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 10) Save 3D screenshots (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option B"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 11) Run viewer
# ==========================================
napari.run()

In [ ]:
################new version opcion B################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    white_tophat, h_maxima
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Pick your file (uncomment the one you want)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            # Unknown unit: assume meters if tiny, else assume µm
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

# Per-voxel metrics
voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale that preserves volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (physical, then converted to voxels)
# ==========================================
# Soma scale and neurite suppression
soma_min_radius_um     = 3.0    # minimal soma radius for seed (µm). ↑ if two somas merge
neurite_tophat_um      = 0.9    # remove bright tubes/thin bridges up to ~this radius (µm)
tophat_strength        = 1.25   # how much of the thin structures to subtract (1.0–1.5)
smooth_sigma_um        = 0.7    # light denoise before thresholding (µm)

# h-maxima for seed stabilization (fraction of soma thickness)
h_fraction_of_radius   = 0.4    # 0.3–0.6; ↑ to merge bumpy peaks within one soma

# Convert to voxel units
soma_min_radius_vox   = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_tophat_vox    = max(1, int(round(neurite_tophat_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
h_vox = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))

print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_tophat_vox={neurite_tophat_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, h_vox={h_vox}")

# ==========================================
# 3) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune if needed
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius in index units (pixels)
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

blob_ids = np.arange(1, len(blobs) + 1)
df = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) CH2: neurite-suppressed segmentation (CELL vs OUTSIDE)
# ==========================================
# Normalize Ch2
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light 3D Gaussian (accounts for anisotropy)
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# ---- Neurite suppression via white-tophat ----
# Extract thin bright structures (neurites) and subtract them
#thin_struct = white_tophat(ch2, selem=ball(neurite_tophat_vox))
thin_struct = white_tophat(ch2, footprint=ball(neurite_tophat_vox))
enhanced = ch2 - tophat_strength * thin_struct
enhanced = np.clip(enhanced, 0.0, 1.0)

# ---- Foreground (neuron) mask via adaptive per-slice threshold ----
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Remove tiny specks; lightly restore soma rims if needed
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))  # tiny fill without bridging

# ---- Thickness map & h-maxima seeding ----
dist = edt(neuron_mask)

# Keep only thick voxels for seed candidate region
thick = dist >= soma_min_radius_vox

# Stabilize peaks within each soma so we get 1 seed per soma (or per nucleus)
# h is in voxels; use ~40% of soma radius
h = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))
maxima = h_maxima(dist, h=h) & thick

# Clean and label seeds
maxima = remove_small_objects(maxima, min_size=500, connectivity=3)
markers = label(maxima, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (h-maxima).")

# ---- Watershed territories across the neuron ----
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("seed voxels (maxima):", int(maxima.sum()))

# ==========================================
# 6) Map lysosomes to (cell / outside) + per-cell IDs
# ==========================================
location_ch2 = []   # "cell" | "outside"
cell_id_list  = []  # watershed territory id (0 if outside/unassigned)

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cid = int(cell_seg[zz, yy, xx]) if n_cells > 0 else 0
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 7) Per-cell (Ch2) volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge volume with lysosome counts per cell
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 8) Visualization
# ==========================================
# Cell (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma seeds (maxima) layer for QA
try:
    seeds_layer = viewer.add_labels(label(maxima).astype(np.uint16), name='Soma seeds (h-maxima)', opacity=0.6)
    seeds_layer.blending = 'translucent_no_depth'
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df["location_ch2"].tolist()) if "location_ch2" in df else np.array([])
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3],
        size=np.clip(blobs[:, 3] * 2, 2, None),
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option B (tophat neurite suppression + h-maxima)"
except Exception:
    pass

# EXTRA: territory stamps (all cells, yellow if contains lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"

except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 9) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # use neurite-suppressed view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow-ish (cell)
            else:
                color = (255, 255, 255)  # white
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 10) Save 3D screenshots (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option B"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 11) EXTRA: Export Original .CZI as MP4 (RAW z-stack)
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which raw channel to export:
    # raw_stack = image        # Ch1 (lysosomes)
    raw_stack = image_2        # Ch2 (neurons)

    raw_u8 = _to_uint8(raw_stack)

    frames = []
    for z in range(raw_u8.shape[0]):
        frame = cv2.cvtColor(raw_u8[z], cv2.COLOR_GRAY2BGR)
        frames.append(frame)

    out_raw = "original_czi_raw.mp4"
    imageio.mimsave(out_raw, frames, fps=8, format="FFMPEG")
    print(f"Saved: {out_raw}")

except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 12) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # LEFT panel (raw). You can switch to Ch1 if preferred:
    # left_u8 = _to_uint8(image)      # Ch1 raw (lysosomes)
    left_u8 = _to_uint8(image_2)      # Ch2 raw (neurons)

    # RIGHT panel base: use the neurite-suppressed view for clarity
    right_base_u8 = _to_uint8(enhanced)

    Z, H, W = left_u8.shape
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=8, format="FFMPEG")

    for z in range(Z):
        # ----- LEFT (raw) -----
        left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)

        # ----- RIGHT (seg overlay on enhanced Ch2) -----
        base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)

        # Overlay cell mask in green
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # Draw lysosome circles (yellow if inside cell, white if outside)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    inside = ('cell_mask' in globals() and cell_mask[z, y, x])
                    color = (255, 255, 0) if inside else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # Concat panels with a thin divider
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg is unavailable
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(left_u8.shape[0]):
            left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W:
                        color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                        cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 13) Run viewer
# ==========================================
napari.run()

In [ ]:
######OPTION C#####################

In [ ]:
# ==========================================
# OPTION C — Tubeness-suppressed Random Walker instance segmentation for Ch2
# - Suppress thin neurites via Sato tubeness and subtract
# - Seed from distance-transform peaks (peak_local_max) with physical spacing
# - Random walker assigns voxels to the nearest soma seed, guided by image edges
# - Keeps your lysosome detection + CSV/video/fig outputs
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log, peak_local_max
from skimage.filters import gaussian, threshold_local, sato
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries, random_walker

# ==========================================
# CONFIG
# ==========================================
# Choose your file:
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove singleton axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Ch1: lysosomes, Ch2: neuron channel
image   = img_ch1
image_2 = img_ch2

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:  # unknown tiny → meters
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)  # isotropic-equivalent linear size
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g} | equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (µm → vox)
# ==========================================
# Soma/thickness scale
soma_min_radius_um      = 3.0     # thickness threshold for soma core; ↑ if two somas merge
min_peak_separation_um  = 2.5     # ≥ this distance between soma seeds
max_soma_seeds          = 64      # safety cap on number of seeds

# Denoise & neurite suppression
smooth_sigma_um         = 0.7     # light denoise before thresholding
tubeness_sigmas_um      = [0.6, 1.0, 1.6]  # scales for Sato tubeness (neurites)
tubeness_strength       = 1.15    # how much to subtract (1.0–1.4); ↑ to suppress more

# Random walker
rw_beta                 = 200.0   # ↑ makes edges harder to cross
rw_mode                 = "cg"    # "cg" faster/lower memory; fallback to "bf" if it fails

# Convert to voxel units
soma_min_radius_vox     = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
min_peak_separation_vox = max(2, int(round(min_peak_separation_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
tubeness_sigmas_vox = [max(0.5, s / lin_equiv_um) for s in tubeness_sigmas_um]

print(f"soma_min_radius_vox={soma_min_radius_vox}, min_peak_separation_vox={min_peak_separation_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, tubeness_sigmas_vox={tubeness_sigmas_vox}")

# ==========================================
# 3) CH1: Lysosome blobs → metrics in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius (px) for 3D LoG
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: denoise + tubeness suppression
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light anisotropic Gaussian
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# Sato tubeness (bright tubes). Subtract to de-emphasize neurites vs somas.
# sato expects intensities in [0,1]; outputs tubeness in [0,1]-ish
tubeness = sato(ch2, sigmas=tubeness_sigmas_vox, black_ridges=False)
enhanced = np.clip(ch2 - tubeness_strength * tubeness, 0.0, 1.0)

# ==========================================
# 6) Foreground mask (neuron) + thickness map
# ==========================================
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Clean small specks; tiny closing to fill soma rims without bridging
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))

# Thickness inside neuron
dist = edt(neuron_mask)

# ==========================================
# 7) Soma seed detection via distance peaks (one per soma)
# ==========================================
coords = peak_local_max(
    dist,
    min_distance=min_peak_separation_vox,
    threshold_abs=soma_min_radius_vox,   # only peaks with enough thickness
    labels=neuron_mask,
    exclude_border=False
)

# Keep strongest peaks if too many
if coords.size > 0:
    peak_vals = dist[coords[:, 0], coords[:, 1], coords[:, 2]]
    order = np.argsort(-peak_vals)
    if len(order) > max_soma_seeds:
        order = order[:max_soma_seeds]
    coords = coords[order]

n_seeds = len(coords)
print(f"Detected {n_seeds} soma seeds (peak_local_max).")

# Build labeled seed volume for random walker:
# 0 = unlabeled; 1 = background; 2..(n_seeds+1) = soma seeds
markers_rw = np.zeros_like(neuron_mask, dtype=np.int32)

# Background seeds: eroded background so seeds are confidently outside neuron
bg_mask = ~neuron_mask
bg_mask = binary_opening(bg_mask, ball(2))
markers_rw[bg_mask] = 1

# Soma seeds
for i, (zz, yy, xx) in enumerate(coords, start=2):
    markers_rw[zz, yy, xx] = i

# Remove empty case early
if n_seeds == 0:
    print("No soma seeds found; falling back to watershed from soma cores.")
    soma_core = dist >= soma_min_radius_vox
    markers = label(soma_core, connectivity=3)
    if markers.max() > 0:
        cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
        cell_mask = cell_seg > 0
    else:
        cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
        cell_mask = np.zeros_like(neuron_mask, dtype=bool)
else:
    # ==========================================
    # 8) Random Walker instance segmentation
    # ==========================================
    try:
        rw_labels = random_walker(
            enhanced, markers_rw, beta=rw_beta, mode=rw_mode, return_full_prob=False
        )
    except Exception as e:
        print(f"random_walker({rw_mode}) failed ({e}); retrying with 'bf' mode.")
        rw_labels = random_walker(
            enhanced, markers_rw, beta=rw_beta, mode="bf", return_full_prob=False
        )

    # Map RW labels: background=1 → 0; soma labels (2..N+1) → 1..N
    rw_labels = rw_labels.astype(np.int32)
    cell_seg = np.where(rw_labels >= 2, rw_labels - 1, 0).astype(np.int32)
    cell_mask = cell_seg > 0

n_cells = int(cell_seg.max())
print(f"Random-walker cells detected: {n_cells}")
print("neuron voxels:", int(neuron_mask.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma seed markers (for QA)
try:
    seed_lab = np.zeros_like(cell_seg, dtype=np.uint16)
    for i, (zz, yy, xx) in enumerate(coords, start=1):
        seed_lab[zz, yy, xx] = i
    viewer.add_labels(seed_lab, name='Soma seeds (RW)', opacity=0.8)
except Exception:
    pass

# Lysosome points colored by location
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option C (tubeness + random walker)"
except Exception:
    pass

# Territory stamps (yellow if contains lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"
except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # neurite-suppressed view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option B"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 11) Run viewer
# ==========================================
napari.run()

In [ ]:
##############NEW VERSION C#################

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log, peak_local_max
from skimage.filters import gaussian, threshold_local, sato
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball
)
from scipy.ndimage import distance_transform_edt as edt
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries, random_walker

# ==========================================
# CONFIG
# ==========================================
# Choose your file:
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove singleton axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Ch1: lysosomes, Ch2: neuron channel
image   = img_ch1
image_2 = img_ch2

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        # Convert meters to µm when needed (many CZIs store in meters)
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:  # unknown tiny → meters
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)  # isotropic-equivalent linear size
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g} | equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (µm → vox)
# ==========================================
# Soma/thickness scale
soma_min_radius_um      = 3.0     # thickness threshold for soma core; ↑ if two somas merge
min_peak_separation_um  = 2.5     # ≥ this distance between soma seeds
max_soma_seeds          = 64      # safety cap on number of seeds

# Denoise & neurite suppression
smooth_sigma_um         = 0.7     # light denoise before thresholding
tubeness_sigmas_um      = [0.6, 1.0, 1.6]  # scales for Sato tubeness (neurites)
tubeness_strength       = 1.15    # how much to subtract (1.0–1.4); ↑ to suppress more

# Random walker
rw_beta                 = 200.0   # ↑ makes edges harder to cross
rw_mode                 = "cg"    # "cg" faster/lower memory; fallback to "bf" if it fails

# Convert to voxel units
soma_min_radius_vox     = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
min_peak_separation_vox = max(2, int(round(min_peak_separation_um / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
tubeness_sigmas_vox = [max(0.5, s / lin_equiv_um) for s in tubeness_sigmas_um]

print(f"soma_min_radius_vox={soma_min_radius_vox}, min_peak_separation_vox={min_peak_separation_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, tubeness_sigmas_vox={tubeness_sigmas_vox}")

# ==========================================
# 3) CH1: Lysosome blobs → metrics in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius (px) for 3D LoG
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: denoise + tubeness suppression
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light anisotropic Gaussian
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# Sato tubeness (bright tubes). Subtract to de-emphasize neurites vs somas.
# sato expects intensities in [0,1]; outputs tubeness in [0,1]-ish
tubeness = sato(ch2, sigmas=tubeness_sigmas_vox, black_ridges=False)
enhanced = np.clip(ch2 - tubeness_strength * tubeness, 0.0, 1.0)

# ==========================================
# 6) Foreground mask (neuron) + thickness map
# ==========================================
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Clean small specks; tiny closing to fill soma rims without bridging
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))

# Thickness inside neuron
dist = edt(neuron_mask)

# ==========================================
# 7) Soma seed detection via distance peaks (one per soma)
# ==========================================
coords = peak_local_max(
    dist,
    min_distance=min_peak_separation_vox,
    threshold_abs=soma_min_radius_vox,   # only peaks with enough thickness
    labels=neuron_mask,
    exclude_border=False
)

# Keep strongest peaks if too many
if coords.size > 0:
    peak_vals = dist[coords[:, 0], coords[:, 1], coords[:, 2]]
    order = np.argsort(-peak_vals)
    if len(order) > max_soma_seeds:
        order = order[:max_soma_seeds]
    coords = coords[order]

n_seeds = len(coords)
print(f"Detected {n_seeds} soma seeds (peak_local_max).")

# Build labeled seed volume for random walker:
# 0 = unlabeled; 1 = background; 2..(n_seeds+1) = soma seeds
markers_rw = np.zeros_like(neuron_mask, dtype=np.int32)

# Background seeds: eroded background so seeds are confidently outside neuron
bg_mask = ~neuron_mask
bg_mask = binary_opening(bg_mask, ball(2))
markers_rw[bg_mask] = 1

# Soma seeds
for i, (zz, yy, xx) in enumerate(coords, start=2):
    markers_rw[zz, yy, xx] = i

# Remove empty case early
if n_seeds == 0:
    print("No soma seeds found; falling back to watershed from soma cores.")
    soma_core = dist >= soma_min_radius_vox
    markers = label(soma_core, connectivity=3)
    if markers.max() > 0:
        cell_seg = watershed(-dist, markers=markers, mask=neuron_mask)
        cell_mask = cell_seg > 0
    else:
        cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
        cell_mask = np.zeros_like(neuron_mask, dtype=bool)
else:
    # ==========================================
    # 8) Random Walker instance segmentation
    # ==========================================
    try:
        rw_labels = random_walker(
            enhanced, markers_rw, beta=rw_beta, mode=rw_mode, return_full_prob=False
        )
    except Exception as e:
        print(f"random_walker({rw_mode}) failed ({e}); retrying with 'bf' mode.")
        rw_labels = random_walker(
            enhanced, markers_rw, beta=rw_beta, mode="bf", return_full_prob=False
        )

    # Map RW labels: background=1 → 0; soma labels (2..N+1) → 1..N
    rw_labels = rw_labels.astype(np.int32)
    cell_seg = np.where(rw_labels >= 2, rw_labels - 1, 0).astype(np.int32)
    cell_mask = cell_seg > 0

n_cells = int(cell_seg.max())
print(f"Random-walker cells detected: {n_cells}")
print("neuron voxels:", int(neuron_mask.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Soma seed markers (for QA)
try:
    seed_lab = np.zeros_like(cell_seg, dtype=np.uint16)
    for i, (zz, yy, xx) in enumerate(coords, start=1):
        seed_lab[zz, yy, xx] = i
    viewer.add_labels(seed_lab, name='Soma seeds (RW)', opacity=0.8)
except Exception:
    pass

# Lysosome points colored by location
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option C (tubeness + random walker)"
except Exception:
    pass

# Territory stamps (yellow if contains lysosomes)
try:
    if n_cells > 0 and cell_seg.max() > 0:
        STAMP_TERRITORIES = True
        MAX_POINTS_PER_CELL = 60
        if STAMP_TERRITORIES:
            rng = np.random.default_rng(42)
            coords_all, texts_all, colors_all = [], [], []
            col_yes = np.array([1.0, 1.0, 0.0, 0.9])    # yellow
            col_no  = np.array([0.8, 0.8, 0.8, 0.85])   # gray

            for cid in range(1, int(cell_seg.max()) + 1):
                zz, yy, xx = np.where(cell_seg == cid)
                if zz.size == 0:
                    continue
                k = min(MAX_POINTS_PER_CELL, zz.size)
                idx = rng.choice(zz.size, size=k, replace=False)
                sample = np.stack([zz[idx], yy[idx], xx[idx]], axis=1)

                coords_all.append(sample)
                texts_all.extend([f"{cid}"] * k)
                colors_all.append(np.tile(col_yes if cid in ids_with_lyso else col_no, (k, 1)))

            if coords_all:
                coords_all = np.concatenate(coords_all, axis=0)
                colors_all = np.concatenate(colors_all, axis=0)
                terr = viewer.add_points(
                    coords_all.astype(float),
                    name="Cell ID territory stamps",
                    size=0.1,
                    face_color=[0, 0, 0, 0],
                    edge_color=[0, 0, 0, 0],
                    edge_width=0
                )
                terr.text = {"text": texts_all, "size": 10, "color": colors_all, "anchor": "center"}
                terr.blending = "translucent_no_depth"
except Exception as e:
    print("Label overlay error:", e)

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # neurite-suppressed view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option B"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 14) EXTRA: Export Original .CZI as MP4 (RAW z-stack)
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which raw channel to export:
    # raw_stack = image        # Ch1 (lysosomes)
    raw_stack = image_2        # Ch2 (neurons)

    raw_u8 = _to_uint8(raw_stack)

    frames = []
    for z in range(raw_u8.shape[0]):
        frame = cv2.cvtColor(raw_u8[z], cv2.COLOR_GRAY2BGR)
        frames.append(frame)

    out_raw = "original_czi_raw.mp4"
    imageio.mimsave(out_raw, frames, fps=8, format="FFMPEG")
    print(f"Saved: {out_raw}")

except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 15) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # LEFT panel (raw). You can switch to Ch1 if preferred:
    # left_u8 = _to_uint8(image)      # Ch1 raw (lysosomes)
    left_u8 = _to_uint8(image_2)      # Ch2 raw (neurons)

    # RIGHT panel base: use neurite-suppressed 'enhanced' for clarity
    right_base_u8 = _to_uint8(enhanced)

    Z, H, W = left_u8.shape
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=8, format="FFMPEG")

    for z in range(Z):
        # ----- LEFT (raw) -----
        left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)

        # ----- RIGHT (seg overlay on enhanced Ch2) -----
        base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)

        # Overlay cell mask in green
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # Draw lysosome circles (yellow if inside cell, white if outside)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    inside = ('cell_mask' in globals() and cell_mask[z, y, x])
                    color = (255, 255, 0) if inside else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # Concat panels with a thin divider
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg is unavailable
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(left_u8.shape[0]):
            left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W:
                        color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                        cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 16) Run viewer
# ==========================================
napari.run()

In [ ]:
##################OPTION D#########################

In [ ]:
# ==========================================
# OPTION D — Opening-by-Reconstruction + Gradient Watershed (3D, anisotropy-aware)
# - Suppress thin neurites via morphological opening-by-reconstruction (OBR)
# - Seed from thick soma cores (distance >= radius threshold, in µm)
# - Watershed on 3D gradient magnitude to split touching somas along edges
# - Keeps your lysosome detection + CSV/video/fig outputs
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    erosion, reconstruction, h_maxima
)
from scipy.ndimage import distance_transform_edt as edt
from scipy.ndimage import gaussian_gradient_magnitude as ggm
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Choose your file (uncomment as needed)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale preserving volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (physical µm → vox)
# ==========================================
# Soma & neurite scales
soma_min_radius_um    = 3.0   # thickness for soma core seeds; ↑ if somas merge
neurite_radius_um     = 1.0   # neurite thickness to remove via OBR; tune 0.6–1.4
smooth_sigma_um       = 0.7   # light denoise before thresholding; 0.5–1.0
grad_sigma_um         = 0.6   # smoothing for gradient magnitude used by watershed
h_fraction_of_radius  = 0.35  # h-maxima height as fraction of soma radius (optional)

# Convert to voxel units
soma_min_radius_vox = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_radius_vox  = max(1, int(round(neurite_radius_um  / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
grad_sigma_vox = (
    max(0.1, grad_sigma_um / vz_um),
    max(0.1, grad_sigma_um / vy_um),
    max(0.1, grad_sigma_um / vx_um),
)
h_vox = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))

print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_radius_vox={neurite_radius_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, grad_sigma_vox={grad_sigma_vox}, h_vox={h_vox}")

# ==========================================
# 3) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # LoG radius in index units (3D)
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um, "y_um": y_um, "x_um": x_um,
    "diameter_um": diameter_um, "radius_um": radius_um, "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: anisotropic denoise + Opening-by-Reconstruction
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light anisotropic Gaussian
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# ---- Opening-by-Reconstruction (OBR) to remove thin bright neurites while preserving soma shapes ----
# 1) Erode with a ball roughly equal to neurite radius; 2) Reconstruct by dilation under the original
seed = erosion(ch2, ball(neurite_radius_vox))
obr = reconstruction(seed, ch2, method='dilation')  # same dtype as ch2, in [0,1]

# OPTION: rescale back to [0,1] (usually already is)
enhanced = np.clip(obr, 0.0, 1.0)

# ==========================================
# 6) Foreground mask + thickness map
# ==========================================
# Adaptive threshold per z-slice (illumination-robust)
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Clean small specks; tiny closing to restore soma rims (won't bridge due to OBR)
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))

# Thickness inside neuron
dist = edt(neuron_mask)

# ==========================================
# 7) Soma seeds from thick cores (+ optional h-maxima stabilization)
# ==========================================
soma_core = dist >= soma_min_radius_vox     # thick voxels only
# Optional: stabilize peaks so 1 seed per soma
maxima = h_maxima(dist, h=h_vox) & soma_core
# If h-maxima yields nothing (rare), fall back to soma_core
seed_region = maxima if maxima.any() else soma_core

# Clean & label seeds
seed_region = binary_opening(seed_region, ball(1))
seed_region = remove_small_objects(seed_region, min_size=1500, connectivity=3)
markers = label(seed_region, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (OBR+thickness).")

# ==========================================
# 8) Gradient-weighted watershed (cuts along edges)
# ==========================================
# Compute 3D gradient magnitude on the OBR-enhanced image
grad = ggm(enhanced, sigma=grad_sigma_vox)  # accepts per-axis sigmas
# Normalize gradient to [0,1] for stability
gmin, gmax = float(grad.min()), float(grad.max())
grad_n = (grad - gmin) / (gmax - gmin) if gmax > gmin else grad*0

if n_cells > 0:
    cell_seg = watershed(
        image=grad_n,            # basins = low gradient; ridges at edges
        markers=markers,
        mask=neuron_mask,
        compactness=0.001,       # gentle compactness to avoid wispy territories
        watershed_line=True      # boundaries labeled as 0
    ).astype(np.int32)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels (seed region):", int(seed_region.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Display seeds for QA
try:
    viewer.add_labels(markers.astype(np.uint16), name='Soma seeds (OBR)', opacity=0.6)
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # OBR-enhanced view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option D (OBR + Gradient Watershed)"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 14) Run viewer
# ==========================================
napari.run()

In [ ]:
##NEW VERSION OPTION D#####

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    erosion, reconstruction, h_maxima
)
from scipy.ndimage import distance_transform_edt as edt
from scipy.ndimage import gaussian_gradient_magnitude as ggm
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Choose your file (uncomment as needed)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove single-dim axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Choose which channel is lysosomes vs neuron channel
image   = img_ch1      # Ch1: lysosome channel
image_2 = img_ch2      # Ch2: neuron (CELL vs OUTSIDE)

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um                 # µm^3 per voxel
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)           # linear scale preserving volume
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g}  |  equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (physical µm → vox)
# ==========================================
# Soma & neurite scales
soma_min_radius_um    = 3.0   # thickness for soma core seeds; ↑ if somas merge
neurite_radius_um     = 1.0   # neurite thickness to remove via OBR; tune 0.6–1.4
smooth_sigma_um       = 0.7   # light denoise before thresholding; 0.5–1.0
grad_sigma_um         = 0.6   # smoothing for gradient magnitude used by watershed
h_fraction_of_radius  = 0.35  # h-maxima height as fraction of soma radius (optional)

# Convert to voxel units
soma_min_radius_vox = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_radius_vox  = max(1, int(round(neurite_radius_um  / lin_equiv_um)))
smooth_sigma_vox = (
    max(0.1, smooth_sigma_um / vz_um),
    max(0.1, smooth_sigma_um / vy_um),
    max(0.1, smooth_sigma_um / vx_um),
)
grad_sigma_vox = (
    max(0.1, grad_sigma_um / vz_um),
    max(0.1, grad_sigma_um / vy_um),
    max(0.1, grad_sigma_um / vx_um),
)
h_vox = max(1, int(round(h_fraction_of_radius * soma_min_radius_vox)))

print(f"soma_min_radius_vox={soma_min_radius_vox}, neurite_radius_vox={neurite_radius_vox}, "
      f"smooth_sigma_vox={smooth_sigma_vox}, grad_sigma_vox={grad_sigma_vox}, h_vox={h_vox}")

# ==========================================
# 3) CH1: blobs (lysosomes) — compute metrics directly in µm
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,   # tune
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # LoG radius in index units (3D)
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um, "y_um": y_um, "x_um": x_um,
    "diameter_um": diameter_um, "radius_um": radius_um, "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: anisotropic denoise + Opening-by-Reconstruction
# ==========================================
# Normalize
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Light anisotropic Gaussian
ch2 = gaussian(vol, sigma=smooth_sigma_vox, preserve_range=True)

# ---- Opening-by-Reconstruction (OBR) to remove thin bright neurites while preserving soma shapes ----
# 1) Erode with a ball roughly equal to neurite radius; 2) Reconstruct by dilation under the original
seed = erosion(ch2, ball(neurite_radius_vox))
obr = reconstruction(seed, ch2, method='dilation')  # same dtype as ch2, in [0,1]

# OPTION: rescale back to [0,1] (usually already is)
enhanced = np.clip(obr, 0.0, 1.0)

# ==========================================
# 6) Foreground mask + thickness map
# ==========================================
# Adaptive threshold per z-slice (illumination-robust)
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    t = threshold_local(R, block_size=61, offset=-0.2*np.std(R))
    neuron_mask[z] = R > t

# Clean small specks; tiny closing to restore soma rims (won't bridge due to OBR)
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)
neuron_mask = binary_closing(neuron_mask, ball(1))

# Thickness inside neuron
dist = edt(neuron_mask)

# ==========================================
# 7) Soma seeds from thick cores (+ optional h-maxima stabilization)
# ==========================================
soma_core = dist >= soma_min_radius_vox     # thick voxels only
# Optional: stabilize peaks so 1 seed per soma
maxima = h_maxima(dist, h=h_vox) & soma_core
# If h-maxima yields nothing (rare), fall back to soma_core
seed_region = maxima if maxima.any() else soma_core

# Clean & label seeds
seed_region = binary_opening(seed_region, ball(1))
seed_region = remove_small_objects(seed_region, min_size=1500, connectivity=3)
markers = label(seed_region, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (OBR+thickness).")

# ==========================================
# 8) Gradient-weighted watershed (cuts along edges)
# ==========================================
# Compute 3D gradient magnitude on the OBR-enhanced image
grad = ggm(enhanced, sigma=grad_sigma_vox)  # accepts per-axis sigmas
# Normalize gradient to [0,1] for stability
gmin, gmax = float(grad.min()), float(grad.max())
grad_n = (grad - gmin) / (gmax - gmin) if gmax > gmin else grad*0

if n_cells > 0:
    cell_seg = watershed(
        image=grad_n,            # basins = low gradient; ridges at edges
        markers=markers,
        mask=neuron_mask,
        compactness=0.001,       # gentle compactness to avoid wispy territories
        watershed_line=True      # boundaries labeled as 0
    ).astype(np.int32)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask, dtype=bool)

print("neuron voxels:", int(neuron_mask.sum()))
print("cell voxels (seed region):", int(seed_region.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Optional boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Display seeds for QA
try:
    viewer.add_labels(markers.astype(np.uint16), name='Soma seeds (OBR)', opacity=0.6)
except Exception:
    pass

# Lysosomes colored by 2-class location (cyan=cell, white=outside)
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # OBR-enhanced view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots (post-segmentation)
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.COLOR_RGBA2BGRA(img_xy))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.COLOR_RGBA2BGRA(img_yz))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.COLOR_RGBA2BGRA(img_xz))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option D (OBR + Gradient Watershed)"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 14) EXTRA: Export Original .CZI as MP4 (RAW z-stack)
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which raw channel to export:
    # raw_stack = image        # Ch1 (lysosomes)
    raw_stack = image_2        # Ch2 (neurons)

    raw_u8 = _to_uint8(raw_stack)

    frames = []
    for z in range(raw_u8.shape[0]):
        frame = cv2.cvtColor(raw_u8[z], cv2.COLOR_GRAY2BGR)
        frames.append(frame)

    out_raw = "original_czi_raw.mp4"
    imageio.mimsave(out_raw, frames, fps=8, format="FFMPEG")
    print(f"Saved: {out_raw}")

except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 15) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # LEFT panel (raw). Switch to Ch1 if preferred:
    # left_u8 = _to_uint8(image)   # Ch1 raw (lysosomes)
    left_u8 = _to_uint8(image_2)   # Ch2 raw (neurons)

    # RIGHT panel base: use OBR-enhanced for clarity
    right_base_u8 = _to_uint8(enhanced)

    Z, H, W = left_u8.shape
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=8, format="FFMPEG")

    for z in range(Z):
        # LEFT (raw)
        left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)

        # RIGHT (seg overlay)
        base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)

        # Overlay cell mask in green
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # Draw lysosomes (yellow if inside cell, white otherwise)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    inside = ('cell_mask' in globals() and cell_mask[z, y, x])
                    color = (255, 255, 0) if inside else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # Concat panels with a narrow divider
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg unavailable
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(left_u8.shape[0]):
            left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])), int(round(b[2]))
                    r = max(2, int(round(b[3])))
                    if 0 <= y < H and 0 <= x < W:
                        color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                        cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 16) Run viewer
# ==========================================
napari.run()

In [ ]:
#####OPTION E######################

In [ ]:
# ==========================================
# OPTION E — Hysteresis + Skeleton-Cut + Distance-Peak Watershed (3D, anisotropy-aware)
# - Edge-preserving median smoothing (anisotropic)
# - Per-slice hysteresis thresholding -> robust neuron foreground
# - Remove neurite bridges by cutting thin skeleton voxels only
# - Seeds from distance-transform local maxima (>= soma thickness)
# - Watershed partitions cells; maps lysosomes to per-cell territories
# ==========================================

import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local, threshold_otsu, apply_hysteresis_threshold
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from scipy.ndimage import distance_transform_edt as edt
from scipy.ndimage import median_filter

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    skeletonize_3d, local_maxima
)
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Choose your file (uncomment as needed)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove singleton axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Ch1: lysosomes, Ch2: neuron channel
image   = img_ch1
image_2 = img_ch2

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)  # isotropic-equivalent linear size
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g} | equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (µm → vox)
# ==========================================
# Smoothing + soma/neurite scales
median_xy_um            = 0.7    # edge-preserving smoothing in XY (µm)
soma_min_radius_um      = 3.0    # thickness threshold for soma cores (µm)  ↑ if somas merge
neurite_max_radius_um   = 1.0    # voxels thinner than this treated as neurite (µm) for skeleton cuts
adaptive_offset_factor  = -0.2   # for threshold_local offset multiplier of slice std

# Hysteresis thresholds (relative to Otsu per-slice)
hyst_low_rel            = 0.70   # low  threshold = low_rel * otsu
hyst_high_rel           = 1.05   # high threshold = high_rel * otsu

# Convert to voxel units
def odd(k): return int(k) + (int(k) + 1) % 2  # ensure odd
median_zyx = (
    1,  # keep Z median very light (anisotropy)
    odd(max(3, round(median_xy_um / vy_um))) if vy_um > 0 else 3,
    odd(max(3, round(median_xy_um / vx_um))) if vx_um > 0 else 3
)
soma_min_radius_vox    = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_max_radius_vox = max(1, int(round(neurite_max_radius_um / lin_equiv_um)))

print(f"median_zyx={median_zyx}, soma_min_radius_vox={soma_min_radius_vox}, "
      f"neurite_max_radius_vox={neurite_max_radius_vox}")

# ==========================================
# 3) CH1: Lysosome blobs → metrics in µm (unchanged)
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius (px) for 3D LoG
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: anisotropic median + hysteresis (slice-wise)
# ==========================================
# Normalize to [0,1]
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Edge-preserving median (heavier in XY than Z to respect anisotropy)
med = median_filter(vol, size=median_zyx)

# Optional local background correction (gentle)
enhanced = np.zeros_like(med, dtype=np.float32)
for z in range(med.shape[0]):
    R = med[z]
    # small local correction + robust offset
    tloc = threshold_local(R, block_size=61, offset=adaptive_offset_factor * np.std(R))
    enhanced[z] = np.clip(R - 0.5 * tloc, 0, 1)

# Hysteresis per-slice using Otsu-derived low/high
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    if np.all(R == 0):
        continue
    otsu = threshold_otsu(R)
    low  = max(0.0, hyst_low_rel  * otsu)
    high = min(1.0, hyst_high_rel * otsu)
    hyst = apply_hysteresis_threshold(R, low=low, high=high)
    neuron_mask[z] = hyst

# Clean small specks
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)

# ==========================================
# 6) Neurite-bridge removal via thin-skeleton cuts
# ==========================================
# Distance inside current mask
dist0 = edt(neuron_mask)

# Skeleton of the mask (1-voxel-wide centerlines)
skel = skeletonize_3d(neuron_mask)

# Only cut skeleton voxels that lie in "thin" regions (≤ neurite_max_radius_vox)
thin = (dist0 <= neurite_max_radius_vox) & neuron_mask
cuts = skel & thin

neuron_mask_cut = neuron_mask.copy()
neuron_mask_cut[cuts] = False

# Light closing to restore soma rims without re-connecting bridges
neuron_mask_cut = binary_closing(neuron_mask_cut, ball(1))

# Recompute distance after cuts
dist = edt(neuron_mask_cut)

# ==========================================
# 7) Seeds from distance peaks (ultimate erosion style)
# ==========================================
# Local maxima of distance (>= soma_min_radius_vox)
seed_candidates = local_maxima(dist)
seed_candidates &= neuron_mask_cut
seed_candidates &= (dist >= soma_min_radius_vox)

# Clean seed candidates & label
seed_candidates = binary_opening(seed_candidates, ball(1))
seed_candidates = remove_small_objects(seed_candidates, min_size=200, connectivity=3)
markers = label(seed_candidates, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (distance peaks).")

# Fallback: if no seeds, try using soma cores directly
if n_cells == 0:
    soma_core = dist >= soma_min_radius_vox
    soma_core = remove_small_objects(soma_core, min_size=1500, connectivity=3)
    markers = label(soma_core, connectivity=3)
    n_cells = int(markers.max())
    print(f"Fallback seeds from soma cores: {n_cells}")

# ==========================================
# 8) Watershed partitioning of neuron territories
# ==========================================
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask_cut)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask_cut, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask_cut, dtype=bool)

print("neuron voxels (pre-cuts):", int(neuron_mask.sum()))
print("neuron voxels (post-cuts):", int(neuron_mask_cut.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask_cut.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Seeds for QA
try:
    viewer.add_labels(markers.astype(np.uint16), name='Soma seeds (dist peaks)', opacity=0.6)
except Exception:
    pass

# Lysosome points colored by location
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option E (hysteresis + skeleton cuts + distance peaks)"
except Exception:
    pass

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # median+background-corrected view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option E"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 14) Run viewer
# ==========================================
napari.run()

In [ ]:
######NEW VERSION OPTION E"""""

In [ ]:
import czifile
import numpy as np
from skimage.feature import blob_log
from skimage.filters import gaussian, threshold_local, threshold_otsu, apply_hysteresis_threshold
import napari
import pandas as pd
import imageio
import cv2
import xml.etree.ElementTree as ET

from scipy.ndimage import distance_transform_edt as edt
from scipy.ndimage import median_filter

from skimage.morphology import (
    remove_small_objects, binary_opening, binary_closing, ball,
    skeletonize_3d, local_maxima
)
from skimage.measure import label
from skimage.segmentation import watershed, find_boundaries

# ==========================================
# CONFIG
# ==========================================
# Choose your file (uncomment as needed)
# file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_1_Airy_010724.czi"
file_path = r"C:/Users/nahue/Downloads/PROYECT OREN/40A_UAS-TMEM1923x-HA x 71G10 40A MARCM_L3_2_Airy_010724.czi"

# ==========================================
# 1) LOAD .CZI IMAGE + VOXEL SIZE (µm)
# ==========================================
with czifile.CziFile(file_path) as czi:
    img = czi.asarray()
    meta_xml = czi.metadata()

img = np.squeeze(img)  # remove singleton axes
print("Raw CZI shape:", img.shape)

# --- Find channel axis ---
if img.shape[0] == 2:        # (C, Z, Y, X)
    img_ch1 = img[0]
    img_ch2 = img[1]
elif img.shape[1] == 2:      # (Z, C, Y, X)
    img_ch1 = img[:, 0]
    img_ch2 = img[:, 1]
else:
    raise RuntimeError(f"Can't auto-detect channel. Image shape: {img.shape}")

# Ch1: lysosomes, Ch2: neuron channel
image   = img_ch1
image_2 = img_ch2

# --- Parse voxel size in µm (fallback = 1 µm) ---
vz_um = vy_um = vx_um = 1.0
try:
    r = ET.fromstring(meta_xml)

    def _get_um(axis: str) -> float:
        v = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}Value")
        u = r.find(f".//{{*}}Scaling/{{*}}Items/{{*}}Distance[@Id='{axis}']/{{*}}DefaultUnit")
        val = float(v.text) if v is not None else 1.0
        unit = (u.text or "").lower() if u is not None else ""
        if unit in ("m", "meter", "metre") or (unit == "" and val < 1e-3):
            val *= 1e6
        elif unit in ("µm", "um", "micrometer", "micrometre"):
            pass
        else:
            if val < 1e-3:
                val *= 1e6
        return val

    vx_um, vy_um, vz_um = _get_um("X"), _get_um("Y"), _get_um("Z")
except Exception as e:
    print("Voxel size not found in metadata; using 1 µm:", e)

voxel_um3 = vz_um * vy_um * vx_um
lin_equiv_um = voxel_um3 ** (1.0 / 3.0)  # isotropic-equivalent linear size
print(f"Voxel size (µm): X={vx_um:.4g}, Y={vy_um:.4g}, Z={vz_um:.4g} | equiv linear µm={lin_equiv_um:.4g}")

# ==========================================
# 2) PARAMETERS (µm → vox)
# ==========================================
# Smoothing + soma/neurite scales
median_xy_um            = 0.7    # edge-preserving smoothing in XY (µm)
soma_min_radius_um      = 3.0    # thickness threshold for soma cores (µm)  ↑ if somas merge
neurite_max_radius_um   = 1.0    # voxels thinner than this treated as neurite (µm) for skeleton cuts
adaptive_offset_factor  = -0.2   # for threshold_local offset multiplier of slice std

# Hysteresis thresholds (relative to Otsu per-slice)
hyst_low_rel            = 0.70   # low  threshold = low_rel * otsu
hyst_high_rel           = 1.05   # high threshold = high_rel * otsu

# Convert to voxel units
def odd(k): return int(k) + (int(k) + 1) % 2  # ensure odd
median_zyx = (
    1,  # keep Z median very light (anisotropy)
    odd(max(3, round(median_xy_um / vy_um))) if vy_um > 0 else 3,
    odd(max(3, round(median_xy_um / vx_um))) if vx_um > 0 else 3
)
soma_min_radius_vox    = max(2, int(round(soma_min_radius_um / lin_equiv_um)))
neurite_max_radius_vox = max(1, int(round(neurite_max_radius_um / lin_equiv_um)))

print(f"median_zyx={median_zyx}, soma_min_radius_vox={soma_min_radius_vox}, "
      f"neurite_max_radius_vox={neurite_max_radius_vox}")

# ==========================================
# 3) CH1: Lysosome blobs → metrics in µm (unchanged)
# ==========================================
image_smooth = gaussian(image, sigma=1)
blobs = blob_log(
    image_smooth,
    min_sigma=1,
    max_sigma=10,
    num_sigma=8,
    threshold=0.003,
    overlap=0.5
)
if len(blobs) > 0:
    blobs[:, 3] = blobs[:, 3] * np.sqrt(3)  # radius (px) for 3D LoG
print(f"Detected {len(blobs)} lysosomes.")

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um   = (blobs[:, 3] * lin_equiv_um) * 0.5
    diameter_um = 2.0 * radius_um
    volume_um3  = (4.0/3.0) * np.pi * (radius_um ** 3)
else:
    z_um = y_um = x_um = np.array([])
    radius_um = diameter_um = volume_um3 = np.array([])

df = pd.DataFrame({
    "id": np.arange(1, len(blobs) + 1),
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "diameter_um": diameter_um,
    "radius_um": radius_um,
    "volume_um3": volume_um3,
})
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions.csv (µm-only)")

# ==========================================
# 4) VIEWER base
# ==========================================
viewer = napari.Viewer()
viewer.add_image(image_2, name='Ch2 raw', blending='additive')
viewer.add_image(image,  name='Ch1 raw', blending='additive')

# ==========================================
# 5) Ch2 preprocessing: anisotropic median + hysteresis (slice-wise)
# ==========================================
# Normalize to [0,1]
vol = image_2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
vol = (vol - vmin) / (vmax - vmin) if vmax > vmin else np.zeros_like(vol, dtype=np.float32)

# Edge-preserving median (heavier in XY than Z to respect anisotropy)
med = median_filter(vol, size=median_zyx)

# Optional local background correction (gentle)
enhanced = np.zeros_like(med, dtype=np.float32)
for z in range(med.shape[0]):
    R = med[z]
    # small local correction + robust offset
    tloc = threshold_local(R, block_size=61, offset=adaptive_offset_factor * np.std(R))
    enhanced[z] = np.clip(R - 0.5 * tloc, 0, 1)

# Hysteresis per-slice using Otsu-derived low/high
neuron_mask = np.zeros_like(enhanced, dtype=bool)
for z in range(enhanced.shape[0]):
    R = enhanced[z]
    if np.all(R == 0):
        continue
    otsu = threshold_otsu(R)
    low  = max(0.0, hyst_low_rel  * otsu)
    high = min(1.0, hyst_high_rel * otsu)
    hyst = apply_hysteresis_threshold(R, low=low, high=high)
    neuron_mask[z] = hyst

# Clean small specks
neuron_mask = remove_small_objects(neuron_mask, min_size=2000, connectivity=3)

# ==========================================
# 6) Neurite-bridge removal via thin-skeleton cuts
# ==========================================
# Distance inside current mask
dist0 = edt(neuron_mask)

# Skeleton of the mask (1-voxel-wide centerlines)
skel = skeletonize_3d(neuron_mask)

# Only cut skeleton voxels that lie in "thin" regions (≤ neurite_max_radius_vox)
thin = (dist0 <= neurite_max_radius_vox) & neuron_mask
cuts = skel & thin

neuron_mask_cut = neuron_mask.copy()
neuron_mask_cut[cuts] = False

# Light closing to restore soma rims without re-connecting bridges
neuron_mask_cut = binary_closing(neuron_mask_cut, ball(1))

# Recompute distance after cuts
dist = edt(neuron_mask_cut)

# ==========================================
# 7) Seeds from distance peaks (ultimate erosion style)
# ==========================================
# Local maxima of distance (>= soma_min_radius_vox)
seed_candidates = local_maxima(dist)
seed_candidates &= neuron_mask_cut
seed_candidates &= (dist >= soma_min_radius_vox)

# Clean seed candidates & label
seed_candidates = binary_opening(seed_candidates, ball(1))
seed_candidates = remove_small_objects(seed_candidates, min_size=200, connectivity=3)
markers = label(seed_candidates, connectivity=3)
n_cells = int(markers.max())
print(f"Detected {n_cells} soma seeds (distance peaks).")

# Fallback: if no seeds, try using soma cores directly
if n_cells == 0:
    soma_core = dist >= soma_min_radius_vox
    soma_core = remove_small_objects(soma_core, min_size=1500, connectivity=3)
    markers = label(soma_core, connectivity=3)
    n_cells = int(markers.max())
    print(f"Fallback seeds from soma cores: {n_cells}")

# ==========================================
# 8) Watershed partitioning of neuron territories
# ==========================================
if n_cells > 0:
    cell_seg = watershed(-dist, markers=markers, mask=neuron_mask_cut)
    cell_mask = cell_seg > 0
else:
    cell_seg = np.zeros_like(neuron_mask_cut, dtype=np.int32)
    cell_mask = np.zeros_like(neuron_mask_cut, dtype=bool)

print("neuron voxels (pre-cuts):", int(neuron_mask.sum()))
print("neuron voxels (post-cuts):", int(neuron_mask_cut.sum()))

# ==========================================
# 9) Map lysosomes to (cell / outside) with per-cell IDs
# ==========================================
location_ch2 = []
cell_id_list  = []

if len(blobs) > 0:
    Z, Y, X = neuron_mask_cut.shape
    for zc, yc, xc in blobs[:, :3]:
        zz, yy, xx = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X):
            location_ch2.append("outside"); cell_id_list.append(0); continue

        if cell_mask[zz, yy, xx]:
            location_ch2.append("cell")
            cell_id_list.append(int(cell_seg[zz, yy, xx]))
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

if len(df) > 0:
    df["location_ch2"] = location_ch2
    df["cell_id_ch2"]  = cell_id_list

    df.groupby("location_ch2").size().reset_index(name="count") \
      .to_csv("lysosome_counts_cell_vs_outside.csv", index=False)

    (df[df["location_ch2"] == "cell"]
        .groupby("cell_id_ch2").size()
        .reset_index(name="count")
        .to_csv("lysosome_counts_by_cell.csv", index=False))

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)

print("Saved: lysosome_counts_cell_vs_outside.csv, lysosome_counts_by_cell.csv, "
      "lysosomes_with_cell_vs_outside.csv (µm-only metrics)")

# For territory stamps later
ids_with_lyso = set()
if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
    hits = df[(df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)]
    ids_with_lyso = set(hits["cell_id_ch2"].astype(int).unique())

# ==========================================
# 10) Per-cell volumes (µm^3)
# ==========================================
cell_volume_df = pd.DataFrame(columns=["cell_id_ch2", "voxel_count", "volume_um3"])
if n_cells > 0 and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3

    cell_volume_df = pd.DataFrame({
        "cell_id_ch2": cell_ids,
        "voxel_count": voxels,
        "volume_um3": vol_um3
    })
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

    # Optional: merge with lysosome counts
    try:
        if len(df) > 0 and "location_ch2" in df and "cell_id_ch2" in df:
            lys_counts = (df[df["location_ch2"] == "cell"]
                          .groupby("cell_id_ch2")
                          .size()
                          .reset_index(name="lysosome_count"))
            merged = (cell_volume_df
                      .merge(lys_counts, on="cell_id_ch2", how="left")
                      .fillna({"lysosome_count": 0}))
            merged.to_csv("cell_metrics_ch2.csv", index=False)
            print("Saved: cell_metrics_ch2.csv")
    except Exception as e:
        print("Merge with lysosome counts failed:", e)

# ==========================================
# 11) Visualization in napari
# ==========================================
# Cell mask (green)
cell_layer = viewer.add_labels(cell_mask.astype(np.uint8), name='Cell (Ch2)', opacity=0.35)
try:
    cell_layer.color = {1: (0.0, 1.0, 0.0, 1.0)}  # green
except Exception:
    pass
cell_layer.blending = 'translucent_no_depth'

# Per-cell territories (labels)
try:
    cellid_layer = viewer.add_labels(
        cell_seg.astype(np.uint16),
        name='Cell ID (Ch2)',
        opacity=0.25
    )
    cellid_layer.blending = 'translucent_no_depth'
    # Boundaries overlay (magenta)
    boundaries = find_boundaries(cell_seg, connectivity=1, mode='outer')
    viewer.add_image(
        boundaries.astype(np.uint8),
        name='Cell ID boundaries',
        blending='additive',
        contrast_limits=(0, 1),
        colormap='magenta',
        opacity=0.6
    )
except Exception:
    pass

# Seeds for QA
try:
    viewer.add_labels(markers.astype(np.uint16), name='Soma seeds (dist peaks)', opacity=0.6)
except Exception:
    pass

# Lysosome points colored by location
if len(blobs) > 0:
    loc = np.array(df.get("location_ch2", []))
    colors = np.zeros((len(loc), 4), dtype=float)
    if loc.size > 0:
        colors[loc == "cell"]    = [0.0, 1.0, 1.0, 1.0]  # cyan
        colors[loc == "outside"] = [1.0, 1.0, 1.0, 1.0]  # white

    pts = viewer.add_points(
        blobs[:, :3] if len(blobs) > 0 else np.empty((0, 3)),
        size=np.clip(blobs[:, 3] * 2, 2, None) if len(blobs) > 0 else 2,
        name='Lysosomes (cell vs outside)'
    )
    try:
        if loc.size > 0:
            pts.face_color = colors
            pts.edge_color = 'black'
            pts.edge_width = 0.3
            pts.properties = {
                'lys_id': df['id'].to_numpy(),
                'where':  df['location_ch2'].to_numpy(),
                'cell':   df['cell_id_ch2'].to_numpy(),
                'diameter_um': df['diameter_um'].to_numpy(),
                'volume_um3': df['volume_um3'].to_numpy()
            }
            pts.text = {'text': 'ID:{lys_id}  C:{cell}', 'size': 10, 'color': 'yellow', 'anchor': 'upper left'}
    except Exception:
        pass

# Title
try:
    viewer.title = "Neuron segmentation — Option E (hysteresis + skeleton cuts + distance peaks)"
except Exception:
    pass

# ==========================================
# 12) Quick fused 2D video (optional)
# ==========================================
img_norm_2 = (enhanced * 255).astype(np.uint8)  # median+background-corrected view
frames_fused = []
Z = img_norm_2.shape[0]
for z in range(Z):
    base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)
    cell = (cell_mask[z].astype(np.uint8) * 255)

    overlay = base.copy()
    overlay[..., 1] = np.maximum(overlay[..., 1], cell)  # green for cell
    overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

    if len(blobs) > 0:
        z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
        for b in z_blobs:
            y, x = int(round(b[1])), int(round(b[2]))
            r = max(2, int(round(b[3])))
            if 0 <= y < cell_mask.shape[1] and 0 <= x < cell_mask.shape[2] and cell_mask[z, y, x]:
                color = (255, 255, 0)  # yellow (inside cell)
            else:
                color = (255, 255, 255)  # white (outside)
            cv2.circle(overlay, (x, y), r, color, 2)

    frames_fused.append(overlay)

try:
    imageio.mimsave('ch2_fused_cell.mp4', frames_fused, fps=8, format='FFMPEG')
    print("Saved: ch2_fused_cell.mp4")
except TypeError:
    imageio.mimsave('ch2_fused_cell.gif', frames_fused, fps=8)
    print("Saved: ch2_fused_cell.gif")

# ==========================================
# 13) Save 3D screenshots
# ==========================================
def save_xy_3d_screenshot(viewer, path='ch2_segmentation_XY_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (90, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 90
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xy = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xy)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xy, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XY screenshot: {path}")

def save_yz_3d_screenshot(viewer, path='ch2_segmentation_YZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 90, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 90
        except Exception:
            pass
    img_yz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_yz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_yz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D YZ screenshot: {path}")

def save_xz_3d_screenshot(viewer, path='ch2_segmentation_XZ_3d.png'):
    viewer.dims.ndisplay = 3
    try:
        viewer.camera.angles = (0, 0, 0)
    except Exception:
        try:
            viewer.camera.elevation = 0
            viewer.camera.azimuth = 0
        except Exception:
            pass
    img_xz = viewer.screenshot(canvas_only=True)
    try:
        imageio.imwrite(path, img_xz)
    except Exception:
        cv2.imwrite(path, cv2.cvtColor(img_xz, cv2.COLOR_RGBA2BGRA))
    print(f"Saved 3D XZ screenshot: {path}")

viewer.dims.ndisplay = 3
try:
    viewer.title = "CELL SEGMENTATION WITH LYSOSOMES — Option E"
except Exception:
    pass

save_xy_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XY_3d.png')
save_yz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_YZ_3d.png')
save_xz_3d_screenshot(viewer, path='cells_segmentation_lysosomes_XZ_3d.png')

# ==========================================
# 14) EXTRA: Export Original .CZI as MP4 (RAW z-stack)
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # Choose which raw channel to export:
    # raw_stack = image        # Ch1 (lysosomes)
    raw_stack = image_2        # Ch2 (neurons)

    raw_u8 = _to_uint8(raw_stack)
    frames = [cv2.cvtColor(raw_u8[z], cv2.COLOR_GRAY2BGR) for z in range(raw_u8.shape[0])]
    imageio.mimsave("original_czi_raw.mp4", frames, fps=8, format="FFMPEG")
    print("Saved: original_czi_raw.mp4")
except Exception as e:
    print("MP4 export of original CZI failed:", e)

# ==========================================
# 15) EXTRA: Side-by-side (RAW | SEGMENTED) MP4
# ==========================================
try:
    def _to_uint8(vol):
        vol = vol.astype(np.float32)
        vmin, vmax = float(vol.min()), float(vol.max())
        if vmax <= vmin:
            return np.zeros_like(vol, dtype=np.uint8)
        return (255.0 * (vol - vmin) / (vmax - vmin)).astype(np.uint8)

    # LEFT panel (raw). Switch to Ch1 if preferred:
    # left_u8 = _to_uint8(image)   # Ch1 raw (lysosomes)
    left_u8 = _to_uint8(image_2)   # Ch2 raw (neurons)

    # RIGHT panel base: use enhanced for clarity
    right_base_u8 = _to_uint8(enhanced)

    Z, H, W = left_u8.shape
    out_name = "raw_vs_segmented_side_by_side.mp4"
    writer = imageio.get_writer(out_name, fps=8, format="FFMPEG")

    for z in range(Z):
        left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
        base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)

        # Overlay cell mask in green
        if 'cell_mask' in globals():
            cell = (cell_mask[z].astype(np.uint8) * 255)
            overlay = base_bgr.copy()
            overlay[..., 1] = np.maximum(overlay[..., 1], cell)
            right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
        else:
            right_bgr = base_bgr

        # Draw lysosomes (yellow if inside cell, white otherwise)
        if 'blobs' in globals() and len(blobs) > 0:
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = max(2, int(round(b[3])))
                if 0 <= y < H and 0 <= x < W:
                    inside = ('cell_mask' in globals() and cell_mask[z, y, x])
                    color = (255, 255, 0) if inside else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

        # Concat panels
        if left_bgr.shape != right_bgr.shape:
            right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                   interpolation=cv2.INTER_NEAREST)
        divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
        frame = cv2.hconcat([left_bgr, divider, right_bgr])

        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Saved: {out_name}")

except Exception as e:
    # Fallback to GIF if ffmpeg unavailable
    try:
        print("FFMPEG writer failed; attempting GIF fallback. Error:", e)
        frames = []
        for z in range(left_u8.shape[0]):
            left_bgr = cv2.cvtColor(left_u8[z], cv2.COLOR_GRAY2BGR)
            base_bgr = cv2.cvtColor(right_base_u8[z], cv2.COLOR_GRAY2BGR)
            if 'cell_mask' in globals():
                cell = (cell_mask[z].astype(np.uint8) * 255)
                overlay = base_bgr.copy()
                overlay[..., 1] = np.maximum(overlay[..., 1], cell)
                right_bgr = cv2.addWeighted(base_bgr, 1.0, overlay, 0.35, 0.0)
            else:
                right_bgr = base_bgr

            if 'blobs' in globals() and len(blobs) > 0:
                z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
                for b in z_blobs:
                    y, x = int(round(b[1])); r = max(2, int(round(b[3])))
                    color = (255, 255, 0) if ('cell_mask' in globals() and cell_mask[z, y, x]) else (255, 255, 255)
                    cv2.circle(right_bgr, (x, y), r, color, 2)

            if left_bgr.shape != right_bgr.shape:
                right_bgr = cv2.resize(right_bgr, (left_bgr.shape[1], left_bgr.shape[0]),
                                       interpolation=cv2.INTER_NEAREST)
            divider = np.full((left_bgr.shape[0], 4, 3), 32, dtype=np.uint8)
            frame = cv2.hconcat([left_bgr, divider, right_bgr])
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        imageio.mimsave("raw_vs_segmented_side_by_side.gif", frames, fps=8)
        print("Saved: raw_vs_segmented_side_by_side.gif")
    except Exception as e2:
        print("Side-by-side export failed:", e2)

# ==========================================
# 16) Run viewer
# ==========================================
napari.run()